In [1]:
#Get functions from external sources
import copy
from collections import deque
import ipywidgets as widgets
import math
import numpy as np
import random
import threading
import time

from hex2rgb import hex2rgb, rgb2hex
import led_driver_neopixel as led
import nfl_scores



In [2]:
#Define some low-level pin variables
pin_config = [
    {'pin': 10, 'num_pixels': 9, 'start_idx': 0},
    {'pin': 10, 'num_pixels': 15, 'start_idx': 9},
    {'pin': 21, 'num_pixels': 9, 'start_idx': 0},
    {'pin': 21, 'num_pixels': 15, 'start_idx': 9},
]
led_handler = led.PixelHandler(pin_config)

In [3]:
# Team Logos
team1_colors=["#ff0000","#858585"]
team2_colors=["#ff0000","#ff6a00"]
score_indicator_color="#0d6600"

idx_0 = int(pin_config[0]['num_pixels']*(0.6))
idx_2 = int(pin_config[2]['num_pixels']*(0.6))

color_array_0 = np.zeros((pin_config[0]['num_pixels'],3))
color_array_0[:,:] = hex2rgb(team1_colors[1])
color_array_0[0:idx_0, :] = hex2rgb(team1_colors[0])
color_array_2 = np.zeros((pin_config[2]['num_pixels'],3))
color_array_2[:,:] = hex2rgb(team2_colors[1])
color_array_2[0:idx_2, :] = hex2rgb(team2_colors[0])

for idx in range(len(color_array_0)):
    color_array_0_tmp = np.roll(color_array_0, idx, axis=0).tolist()
    color_array_2_tmp = np.roll(color_array_2, idx, axis=0).tolist()
    led_handler.set_color_array(0, color_array_0_tmp, invert=False)
    led_handler.set_color_array(2, color_array_2_tmp, invert=False)
    time.sleep(0.05)

led_handler.set_color_array(0, color_array_0, invert=False)
led_handler.set_color_array(2, color_array_2, invert=False)

In [4]:
color = widgets.ColorPicker(
    concise=False,
    description='Pick a color',
    value='#FFFFFF',
    disabled=False,
    continuous_update=True,
)
output = widgets.Output()
display(color, output)

color_array_0 = np.zeros((pin_config[0]['num_pixels'],3))
color_array_0[:,:] = hex2rgb("#FFFFFF")

def apply_color(new_color, animate="cascade"):
    if animate=='cascade':
        for idx in range(len(color_array_0)):
            color_array_0[idx, :] = hex2rgb(new_color)
            led_handler.set_color_array(0, color_array_0.tolist(), invert=False)
            led_handler.set_color_array(2, color_array_0.tolist(), invert=False)
            time.sleep(0.02)
    elif animate=='fade':
        old_color_array = np.array(color_array_0)
        new_color_array = np.array(color_array_0)
        new_color_array[:,:] = hex2rgb(new_color)
        num_steps = 5
        inc = (new_color_array - old_color_array) / num_steps
        for idx in range(num_steps-1):
            curr_color_array = old_color_array + (idx+1)*(inc)
            led_handler.set_color_array(0, curr_color_array.tolist(), invert=False)
            led_handler.set_color_array(2, curr_color_array.tolist(), invert=False)
            time.sleep(0.01)
    
    if not animate=='cascade':
        color_array_0[:,:] = hex2rgb(new_color)
        led_handler.set_color_array(0, color_array_0.tolist(), invert=False)
        led_handler.set_color_array(2, color_array_0.tolist(), invert=False)
        
apply_color("#FFFFFF", animate='fade')

def on_value_change(change):
    with output:
        print(change['new'])
    
    apply_color(change['new'], animate=None)
    

color.observe(on_value_change, names='value')

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [5]:
main_color = [255,255,255]
main_brightness = 100

def on_slider(red=None, green=None, blue=None, brightness=None):
    global main_brightness
    global main_color
    if red is not None:
        main_color[0] = red['new']
    elif green is not None:
        main_color[1] = green['new']
    elif blue is not None:
        main_color[2] = blue['new']
        
    elif brightness is not None:
        main_brightness = brightness['new']
        
#     print(main_color)
    
    new_bright = np.array(main_color)*main_brightness/100.0
    hex = rgb2hex(tuple([int(x) for x in new_bright]))
    
    color_picker.value="#{}".format(hex)
        
    apply_color(hex, animate='fade')
    
def on_color_picker(change):   
    main_color = list(hex2rgb(change['new']))

red = widgets.IntSlider(
    description='Red',
    continuous_update=True,
    min=0, max=255, step=1,
    value=255,
)

green = widgets.IntSlider(
    description='Green',
    continuous_update=True,
    min=0, max=255, step=1,
    value=255,
)

blue = widgets.IntSlider(
    description='Blue',
    continuous_update=True,
    min=0, max=255, step=1,
    value=255,
)

brightness = widgets.IntSlider(
    description='Brightness',
    continuous_update=True,
    min=0, max=100, step=1,
    value=100,
)


color_picker = widgets.ColorPicker(
    concise=False,
    description='Pick a color',
    value='#FFFFFF',
    disabled=False,
    continuous_update=True,
)

#l = widgets.jslink((red, 'value'), (green, 'value'))

display(color_picker, red, green, blue, brightness)

red.observe(lambda x: on_slider(red=x), names='value')
green.observe(lambda x: on_slider(green=x), names='value')
blue.observe(lambda x: on_slider(blue=x), names='value')
brightness.observe(lambda x: on_slider(brightness=x), names='value')
color_picker.observe(on_color_picker, names='value')

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [50]:
import astral.geocoder
import astral.location
import astral.sun
def get_auto_brightness(date, max_brightness=1.0, min_brightness=0.0):
    # Get the sunrise/sunset data calculator:
    city = astral.geocoder.lookup("Boston", astral.geocoder.database())
    boston = astral.location.Location(city)
    sun_events = astral.sun.sun(city.observer, date=date, tzinfo=boston.timezone)

    # Compute the dawn and dusk ratios
    dawn_ratio = (sun_events['dawn'].hour*60 + sun_events['dawn'].minute)/(60*24)
    dusk_ratio = (sun_events['dusk'].hour*60 + sun_events['dusk'].minute)/(60*24)
    
    now_ratio = ((date.hour*60.0)+date.minute)/(60*24)
    
    
    if (now_ratio <= dawn_ratio) or (now_ratio >= dusk_ratio):
        brightness =  min_brightness
    else:
        middle = (dawn_ratio + dusk_ratio)/2
        span = dusk_ratio - dawn_ratio
        if now_ratio < middle:
            brightness = min_brightness + (max_brightness - min_brightness)*(now_ratio-dawn_ratio)/(span*0.5)
        else:
            brightness = min_brightness + (max_brightness - min_brightness)*(dusk_ratio-now_ratio)/(span*0.5)
            
    return brightness

In [57]:
#Clock

#Get functions from external sources
import copy
import datetime
import numpy as np
import time

from hex2rgb import hex2rgb, rgb2hex
import led_driver_neopixel as led    
        

#Define some low-level pin variables
pin_config = [
    {'pin': 10, 'num_pixels': 9, 'start_idx': 0},
    {'pin': 10, 'num_pixels': 15, 'start_idx': 9},
    {'pin': 21, 'num_pixels': 9, 'start_idx': 0},
    {'pin': 21, 'num_pixels': 15, 'start_idx': 9},
]
led_handler = led.PixelHandler(pin_config)

duration = 20.0 # [sec]
time_start = datetime.datetime.now()
now = datetime.datetime.now()

color_check_time = 60.0 #[sec]
last_color_check = time_start-datetime.timedelta(seconds=color_check_time+1)

week_colors =[
    [255, 150, 0], # Monday
    [255, 60, 0], # Tuesday
    [255, 10, 0], # Wednesday
    [130, 0, 255], # Thursday
    [0, 50, 255], # Friday
    [0, 255, 40], # Saturday
    [100, 255, 0], # Sunday
]

main_color = [255,255,255]
main_brightness = 100

indicator_hr = led.Indicator(
    pin_config[0]['num_pixels'],
)

indicator_min = led.Indicator(
    pin_config[1]['num_pixels'],
    2,
    pin_config[1]['num_pixels']-2,
)

indicator_sec = led.Indicator(
    pin_config[3]['num_pixels'],
    2,
    pin_config[3]['num_pixels']-2,
)

indicator_hr.buffer_color = main_color
indicator_min.buffer_color = [0,0,0]
indicator_sec.buffer_color = [0,0,0]

while (now - time_start).total_seconds() <duration:
    now = datetime.datetime.now()
    
    # Update the color if the weekday changes
    if (now - last_color_check).total_seconds() >= color_check_time:
        weekday = int(now.weekday())
        main_color = week_colors[weekday]
        
        main_brightness = get_auto_brightness(now, 0.80, 0.05)*100
        
        last_color_check = copy.deepcopy(now)
    
    indicator_hr.brightness = main_brightness
    indicator_min.brightness = main_brightness
    indicator_sec.brightness = main_brightness
    
    color_array_hr = indicator_hr.compute_fractional_colors(
        main_color,
        [0,0,0],
        (now.hour+1)/24.0,
    )
    
    color_array_min = indicator_min.compute_fractional_colors(
        main_color,
        [0,0,0],
        (now.minute+1)/60.0,
    )
    
    color_array_sec = indicator_sec.compute_fractional_colors(
        main_color,
        [0,0,0],
        (((now.second)*1e6)+now.microsecond)/60e6,
    )
    
    led_handler.set_color_array(0, color_array_hr, invert=False)
    led_handler.set_color_array(1, color_array_min, invert=True)
    led_handler.set_color_array(3, color_array_sec, invert=True)
    
    time.sleep(0.1)



In [7]:
now.second
(((now.second)*1e6)+now.microsecond)/60e6

0.8961090166666666

In [53]:
main_brightness

9.375000000000018

In [31]:
import astral.geocoder
import astral.location
import astral.sun
import datetime

def get_dawn_dusk_ratios(date):
    # Get the Sunrise/Sunset data calculator:
    city = astral.geocoder.lookup("Boston", astral.geocoder.database())
    boston = astral.location.Location(city)
    sun_events = astral.sun.sun(city.observer, date=date, tzinfo=boston.timezone)

    dawn_ratio = (sun_events['dawn'].hour*60 + sun_events['dawn'].minute)/(60*24)

    dusk_ratio = (sun_events['dusk'].hour*60 + sun_events['dusk'].minute)/(60*24)

    return (dawn_ratio, dusk_ratio)

get_dawn_dusk_ratios(datetime.datetime.now())

(0.24791666666666667, 0.7479166666666667)

In [17]:
s

{'dawn': datetime.datetime(2024, 2, 25, 10, 57, 58, 920865, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 'sunrise': datetime.datetime(2024, 2, 25, 11, 26, 50, 530381, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 'noon': datetime.datetime(2024, 2, 25, 16, 57, 16, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 'sunset': datetime.datetime(2024, 2, 25, 22, 28, 21, 853093, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')),
 'dusk': datetime.datetime(2024, 2, 25, 22, 57, 15, 590142, tzinfo=zoneinfo.ZoneInfo(key='Europe/London'))}

In [55]:
bright = []
for hr in range(24):
    now = datetime.datetime(2024, 2, 25, hr, 0, 0)
    bright.append(get_auto_brightness(now))

for idx, val in enumerate(bright):
    print(idx, val)

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.008333333333333304
7 0.17500000000000004
8 0.34166666666666656
9 0.5083333333333333
10 0.675
11 0.8416666666666666
12 0.9916666666666667
13 0.8250000000000002
14 0.6583333333333332
15 0.4916666666666667
16 0.3250000000000002
17 0.15833333333333321
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
